# O PROBLEMA

Você foi contratado(a) para uma consultoria, e seu trabalho envolve analisar os dados de preço do petróleo brent, que pode ser encontrado no site do ipea. Essa base de dados histórica envolve duas colunas: data e preço (em dólares).

Um grande cliente do segmento pediu para que a consultoria desenvolvesse um dashboard interativo e que gere insights relevantes para tomada de decisão. Além disso, solicitaram que fosse desenvovido um modelo de Machine Learning para fazer o forecasting do preço do petróleo.

Seu objetivo é:

- Criar um dashboard interativo com ferramentas de sua escolha.
- Seu dashboard deve fazer parte de um storytelling que traga insights relevantes sobre a variação do preço do petróleo, como situações geopolíticas, crises econômicas, demanda global por energia e etc. Isso pode te ajudar com seu modelo. É obrigatório que você traga pelo menos 4 insights neste desafio.
- Criar um modelo de Machine Learning que faça a previsão do preço do petróleo diariamente (lembre-se de time series). Esse modelo deve estar contemplado em seu storytelling e deve conter o código que você trabalhou, analisando as performances do modelo.
- Criar um plano para fazer o deploy em produção do modelo, com as ferramentas que são necessárias.
- Faça o MVP do seu modelo em produção usando o Streamlit.

Link para a base de dados: http://www.ipeadata.gov.br/ExibeSerie.aspx?module=m&serid=1650971490&oper=view

Referências: 

https://biblioteca.ibp.org.br/pt-BR/search/23350?exp=brent

https://www.ipea.gov.br/desafios/index.php?option=com_content&view=article&id=2083:catid=28&Itemid=23

## Metodologia

<img src = 'img/CRISP-DM.png'>

Para o desenvolvimento deste projeto, optou por seguir a metodologia CRISP-DM (CRoss Industry Standard Process for Data Mining), largamente utilizada em projetos de dados. A CRISP-DM é composta por 6 etapas, são elas: 

- Análise do negócio (business understanding): essa etapa dedica-se ao entendimento do negócio, isto é, sua produto/serviço, público alvo e quais são as estratégias do setor;

- Análise dos dados (data understanding): uma vez realizada a etapa anterior, inicia-se o entendimento dos dados, que compreende a seleção daqueles que são uteis à resolução da problemática requerida, bem como seu estado (fontes, formatos, etc);

- Preparação dos dados (data preparation): essa etapa dedica-se ao pré-processamento dos dados, em acordo com o que é solicitado pelas soluções que se pretende implementar;

- Modelagem (modeling): a modelagem é a etapa na qual se extrai informações de valor sobre os dados, possibilitando a geração de insights úteis ao negócio, capazes de solucionar a problemática;

- Avaliação (evaluation): momento de avaliação do desempenho do modelo aplicado, isto é, verifica-se se esse respondeu as questões levantadas satisfatoriamente;

- Implementação (deployment): por fim, a implementação é a etapa em que se disponibiliza os resultados e insights obtidos as partes interessadas, comumente por meio de uma ferramenta de data viz ou relatório.

## Análise do negócio

A cotação do petróleo é definida por seu preço, em um determinado momento no mercado onde está sendo negociado. Desse modo, a cotação do petróleo é o resultado da oferta e demanda da commodity no mercado internacional. Nesse caso, a unidade de medida utilizada é de dólares por barril de petróleo. O petróleo Brent é todo o petróleo extraído no Mar do Norte e comercializado na Bolsa de Londres e foi batizado assim porque era extraído de uma base da Shell chamada Brent.

O Brent Crude Oil e o West Texas Intermediate (WTI) são as duas maiores referências comerciais da commodity e a principal diferença entre ambas é originada de seus campos de extração. Enquanto petróleo Brent vem do Mar do Norte, o petróleo WTI encontra-se nos Estados Unidos, sendo negociado, historicamente, com valor menor em relação ao Brent.

A causa principal para essa divergência de valores é que eles são um retrato instantâneo dos preços do petróleo de lugares diferentes: o WTI reflete o mercado dos Estados Unidos e o Brent o do Mar do Norte e Europa. Houve um período em que os dois eram referências igualmente válidas para o preço mundial do petróleo. Contudo, à medida que a produção americana aumentou, o WTI começou a ser afetado pelo surpreendente excesso de oferta nos EUA, que se mostrou maior do que os oleodutos existentes no país podiam suportar.

O petróleo bruto é classificado como leve, médio ou pesado com relação à densidade do líquido, ou como "doce" ou "ácido" com base no conteúdo de enxofre. O petróleo leve e doce - que flutua na água e tem baixo teor de enxofre - é valorizado porque exige um processamento menor para ser convertido em derivados. O WTI e o Brent são tipos leves e doces de petróleo bruto.

Estima-se que mais da metade de todo petróleo do mundo é cotado em termos de petróleo Brent. Sua popularidade vem do fato de ser mais barato, tendo em vista que os poços de petróleo Brent se encontram no mar, o que facilita sua extração em comparação ao WTI, que precisa ser transportado por ferrovias.

O petróleo é negociado de duas formas: mercado à vista e mercado futuro. No mercado à vista, as operações de compra e venda são determinadas com base na oferta e demanda do momento. Uma vez realizada a operação, a liquidação ocorre quase que instantaneamente. Já no mercado futuro, que negocia os chamados “barris de papel”, o petróleo é negociado no “papel”, isto é, com base em um valor monetário de cotação do petróleo, sem que haja a troca física do produto. Os contratos de papel são comprados e vendidos com base nas condições de mercado esperadas em períodos futuros. Os compradores e vendedores no mercado de futuros são investidores, que não possuem qualquer intenção de tomar posse do petróleo real.

A intenção dos investidores é comprar contratos futuros de petróleo, baseados nas cotações WTI e Brent, para fins de lucratividade do capital, adivinhando corretamente se os preços dos barris irão aumentar ou diminuir no futuro. Além disso, alguns produtores utilizam o mercado futuro para se proteger da volatilidade dos preços, travando receitas, que visam cobrir custos e garantir lucros da futura produção de petróleo. As refinarias, por exemplo, compram contratos futuros para travar os custos de compra, tendo em vista uma possível queda do petróleo.

O preço do petróleo é influenciado por fatores como mudanças no valor do dólar, mudanças nas políticas da Organization of Petroleum Exporting Countries (OPEC), mudanças nos níveis de produção e estoque do petróleo, saúde da economia global e implementação ou colapso de acordos internacionais.

## Análise dos dados

Os dados utilizados neste projeto estão disponibilizados em uma tabela no site do IPEA (Instituto de Pesquisa Econômica Aplicada) e serão importados por meio da biblioteca pandas do python. A tabela disponibiliza os preços por barril do petróleo bruto tipo Brent, não incluindo despesa de frete e seguro. Os dados serão então exportados para um arquivo em formato csv, a partir do qual serão trabalhados, evitando problemas de indisponibilidade na fonte.

## Preparação dos dados

### Importando bibliotecas

In [1]:
import warnings
warnings.filterwarnings(action = 'ignore')

In [2]:
import pandas as pd
import numpy as np
import psycopg2 as ps
import pandas.io.sql as sqlio
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

### Importando dados

**Importando dados do banco de dados**

In [3]:
conn = ps.connect(
    dbname = 'postech',
    user ='postgres',
    password = 'postgres',
    host = 'localhost',
    port = '5432'
)

In [4]:
sql = 'SELECT * FROM ipea.preco_brent ORDER BY data DESC'
df = sqlio.read_sql_query(sql, conn) #importando dados do PostgreSQL para o DataFrame pandas
df.head()

,data,preco
0,2023-12-26,80.23
1,2023-12-22,80.23
2,2023-12-21,80.73
3,2023-12-20,81.10
4,2023-12-19,79.82


In [5]:
conn.close()

### Tratando dados

In [6]:
def renomeiaColunas(df):
    df.rename(columns={0: 'data', 1: 'preco'}, inplace=True)
    return df

In [7]:
def tipaDados(df):
    df['data'] = pd.to_datetime(df['data'], format='%d-%m-%Y')
    return df

In [8]:
def excluiDadosNulos(df):
    if 0 not in df.isna().sum().values: 
        print(f'{df.isna().sum().values.sum()} dados nulos excluídos')
        df = df.dropna()
    else:
        print('Não há dados nulos')
    return df

In [9]:
#excluindo dados nulos
df = excluiDadosNulos(df)
df.head()

Não há dados nulos


,data,preco
0,2023-12-26,80.23
1,2023-12-22,80.23
2,2023-12-21,80.73
3,2023-12-20,81.10
4,2023-12-19,79.82


In [10]:
#renomeando colunas
df = renomeiaColunas(df)
df.head()

,data,preco
0,2023-12-26,80.23
1,2023-12-22,80.23
2,2023-12-21,80.73
3,2023-12-20,81.10
4,2023-12-19,79.82


In [11]:
#alterando tipo da coluna de data
df = tipaDados(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11074 entries, 0 to 11073
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   data    11074 non-null  datetime64[ns]
 1   preco   11074 non-null  float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 173.2 KB


### EDA

In [12]:
df_eda = df.sort_values('data', ascending=False).set_index('data')
df_eda.head()

,preco
data,
2023-12-26,80.23
2023-12-22,80.23
2023-12-21,80.73
2023-12-20,81.10
2023-12-19,79.82


In [13]:
#configurando template do plotly
template = 'ggplot2'

**Estatísticas descritivas**

In [14]:
df_eda.describe()

,preco
count,11074.000000
mean,52.754036
std,33.240415
min,9.100000
25%,20.385000
50%,47.805000
75%,75.547500
max,143.950000


**Distribuição das variáveis**

In [15]:
fig = px.histogram(
    data_frame=df_eda, 
    x='preco', 
    nbins=20,
    template=template,
    color_discrete_sequence=['#00008B']
)
fig.update_layout(
    title='Histograma do Preço do Petróleo Brent',
    xaxis_title='Preço',
    yaxis_title='Frequência',
    width=1600, 
    height=600,
    margin=dict(l=100, r=20, t=80, b=80)
)
fig.show()

In [16]:
df_log = np.log(df_eda)

fig = px.histogram(
    data_frame=df_log, 
    x='preco', 
    nbins=20,
    template=template,
    color_discrete_sequence=['#00008B']
)
fig.update_layout(
    title='Histograma do Preço do Petróleo Brent',
    xaxis_title='Preço',
    yaxis_title='Frequência',
    width=1600, 
    height=600,
    margin=dict(l=100, r=20, t=80, b=80)
)
fig.show()

In [17]:
fig = px.box(
    data_frame=df_eda, 
    x = 'preco',
    template = template,
    color_discrete_sequence = ['#00008B']
)
fig.update_layout(
    title = 'Boxplot do Preço do Petróleo Brent',
    xaxis_title = 'Preço (US$)',
    yaxis_title = 'Volume',
    width = 1600, 
    height = 600,
    margin = dict(l = 80, r = 20, t = 80, b = 80),
    template = template
)
fig.show()

#### Analisando a série

In [18]:
from statsmodels.tsa.seasonal import seasonal_decompose #lib para decompor série temporal

In [19]:
resultados = seasonal_decompose(df_eda, period=5)

In [20]:
media_movel = resultados.trend.to_frame()
sazonalidade = resultados.seasonal.to_frame()
residuo = resultados.resid.to_frame()

fig = make_subplots(rows = 4, cols = 1)
fig.add_trace(go.Scatter(x=df_eda.index, y=df.preco, name='Série', marker=dict(color='#00008B')), row=1, col=1)
fig.add_trace(go.Scatter(x=media_movel.index, y=media_movel.trend, name='Media móvel', marker=dict(color='#B22222')), row=2, col=1)
fig.add_trace(go.Scatter(x=sazonalidade.index, y=sazonalidade.seasonal, name='Sazonalidade', marker = dict(color='#008000')), row=3, col=1)
fig.add_trace(go.Scatter(x=residuo.index, y=residuo.resid, name='Resíduo', marker=dict(color='gold')), row=4, col=1)
fig.update_layout(
    title={
        'text': 'Decomposição da Série de Preços do Petróleo Brent',
        'x': 0.5,
        'xanchor': 'center'
    },
    width=1600, 
    height=1200,
    template=template
)
fig.show()

**Analisando sazonalidade**

In [21]:
df_eda = df_eda.loc[df_eda.index >= '2023-01-01']
resultados_teste = seasonal_decompose(df_eda, period=5)
sazonalidade_teste = resultados_teste.seasonal.to_frame()

fig = px.line(
    sazonalidade_teste,
    template=template,
    color_discrete_sequence=['#008000']
)
fig.update_layout(
    title='Sazonalidade da Série de Preços do Petróleo Brent',
    xaxis_title='Período',
    yaxis_title='Sazonalidade',
    showlegend=False,
    width=1600, 
    height=600,
    margin=dict(l=100, r=20, t=80, b=80)
)
fig.show()

## Modelagem

### Statsforecast

É um pacote de modelos probabilísticos (nos dá um intervalo de valores para o que estamos tentando prever) e de ponto (estamos cravando o resultado, dando apenas o valor mais provável).
Modelo estatístico de série univariadas, ou seja, utilizamos apenas a nossa serie target (dependente) para fazer previsões.
Não fica limitado a prever uma série de cada vez, pois ela é escalável para prever múltiplas series ao mesmo tempo.

**Tratando DataFrame para a biblioteca StatsForecast**

In [22]:
df_statsforecast = df[['data', 'preco']].rename(columns={'data': 'ds', 'preco': 'y'})
df_statsforecast['unique_id'] = 'Preco'
df_statsforecast.dropna(inplace=True)
df_statsforecast.head()

,ds,y,unique_id
0,2023-12-26,80.23,Preco
1,2023-12-22,80.23,Preco
2,2023-12-21,80.73,Preco
3,2023-12-20,81.10,Preco
4,2023-12-19,79.82,Preco


**Separando base de treino e de teste**

In [23]:
treino_arima = df_statsforecast.loc[(df_statsforecast['ds'] >= '2000-01-01') & (df_statsforecast['ds'] <= '2023-12-18')] #dados de treino
teste_arima = df_statsforecast.loc[(df_statsforecast['ds'] > '2023-12-18') & (df_statsforecast['ds'] <= '2023-12-26')] #dados de validação (5 dias), quanto maior o período de predição, maior a largura de banda (maior o erro)
h = teste_arima['ds'].nunique() #datas distintas no intervalo (5 dias)
h

5

In [24]:
teste_arima.shape, treino_arima.shape

((5, 3), (7869, 3))

### ARIMA

Se combinarmos a diferenciação com a autorregressão e um modelo de média móvel, obtemos um modelo ARIMA não sazonal. ARIMA é um acrônimo para Média Móvel Integrada AutoRegressiva (neste contexto, “integração” é o inverso de diferenciação). O modelo completo pode ser escrito como

### $$y'_{t} = c + \phi_{1}y'_{t-1} + \cdots + \phi_{p}y'_{t-p} + \theta_{1}\varepsilon_{t-1} + \cdots + \theta_{q}\varepsilon_{t-q} + \varepsilon_{t},$$

onde $y't$ é a série diferenciada (pode ter sido diferenciada mais de uma vez).

Os “preditores” no lado direito incluem ambos os valores defasados ​​de $yt$ e erros atrasados. Chamamos isso de modelo ARIMA($p, d, q$) modelo, onde:

- $p$ = ordem da parte autoregressiva;

- $d$ = grau de primeira diferenciação envolvido;

- $q$ = ordem da parte da média móvel.

As mesmas condições de estacionariedade e invertibilidade usadas para modelos autoregressivos e de média móvel também se aplicam a um modelo ARIMA.



In [25]:
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA

In [26]:
modelo_arima = StatsForecast(models=[AutoARIMA(season_length=5)], freq='B', n_jobs=-1)
modelo_arima.fit(treino_arima)
forecast_arima = modelo_arima.predict(h=h, level=[90])
forecast_arima.ds = teste_arima.ds.to_list()
forecast_arima = forecast_arima.reset_index().merge(teste_arima, on =['ds', 'unique_id'], how='left')
forecast_arima

,unique_id,ds,AutoARIMA,AutoARIMA-lo-90,AutoARIMA-hi-90,y
0,Preco,2023-12-26,78.471909,75.767075,81.176743,80.23
1,Preco,2023-12-22,78.075272,74.561371,81.589172,80.23
2,Preco,2023-12-21,78.151497,74.238960,82.064041,80.73
3,Preco,2023-12-20,78.228195,73.919800,82.536591,81.10
4,Preco,2023-12-19,78.214340,73.503639,82.925041,79.82


In [27]:
#gerando linha contínua para banda de confiança (é preciso uma linha contínua para aplicar o preenchimento)
banda_sup_arima = forecast_arima['AutoARIMA-hi-90'].tolist() #limite superior
banda_inf_arima = forecast_arima['AutoARIMA-lo-90'].tolist() #limite inferior
banda_arima = banda_sup_arima + banda_inf_arima[::-1] #concatenando banda superior e banda inferior invertida 
banda_arima_index = forecast_arima.ds.tolist()
banda_arima_index = banda_arima_index + banda_arima_index[::-1] #concatenando datas em ordem crescente com ordem decrescente 

In [28]:
#concatenando treino e teste
treino_px = treino_arima.drop(columns = 'unique_id').set_index('ds') 
treino_px = treino_px[(treino_px.index >= '2023-09-01') & (treino_px.index < '2023-11-18')]
teste_px = teste_arima.drop(columns = 'unique_id').set_index('ds')
base_arima_px = pd.concat([teste_px, treino_px]).rename(columns = {'y': 'Preço'})
base_arima_px.head()

,Preço
ds,
2023-12-26,80.23
2023-12-22,80.23
2023-12-21,80.73
2023-12-20,81.10
2023-12-19,79.82


In [29]:
fig = px.line(
    base_arima_px,
    template=template,
    color_discrete_sequence=['#00008B']
)
fig.add_scatter(
    x=banda_arima_index, 
    y=banda_arima,
    name='Banda de confiança',
    mode='lines',
    line_color='#B22222',
    line_width=0,
    opacity=0.5,
    fill='toself'
)
fig.add_scatter(
    x=forecast_arima.ds, 
    y=forecast_arima['AutoARIMA'], 
    name='Preço previsto',
    marker=dict(color='#B22222')
)
fig.update_layout(
    title='ARIMA',
    xaxis_title='Período',
    yaxis_title='Preço do Petróleo Brent',
    legend_title='Legenda',
    width=1600, 
    height=600,
    margin=dict(l=100, r=20, t=80, b=80)
)
fig.show()

### Prophet

O Prophet é um modelo produzido pelo Facebook com o objetivo de prever valores futuros com base em dados históricos. Ele funciona identificando padrões nos dados, como tendências que aumentam ou diminuem ao longo do tempo e combinando tendências, sazonalidades e efeitos em uma equação matemática.

### $$y_t = g(t) + s(t) + h(t) + \varepsilon_t,$$ 

onde:

- $g(t)$ = tendência linear ao longo do tempo;

- $s(t)$ = padrões sazonais;

- $h(t)$ = efeitos de feriados;

- $ε$ = termo de erro em forma de ruído.

In [30]:
from prophet import Prophet

In [31]:
treino_prophet = treino_arima.reset_index(drop=True).drop('unique_id', axis=1)
teste_prophet = teste_arima.drop('unique_id', axis=1).set_index('ds', drop=True)
h

5

In [32]:
modelo_prophet = Prophet()
modelo_prophet.fit(treino_prophet)

22:35:51 - cmdstanpy - INFO - Chain [1] start processing
22:36:05 - cmdstanpy - INFO - Chain [1] done processing


In [33]:
future = modelo_prophet.make_future_dataframe(periods=7, include_history=False)
future = future.iloc[[0, 1, 2, 3, 6]].reset_index(drop=True) #excluindo final de semana
future

,ds
0,2023-12-19
1,2023-12-20
2,2023-12-21
3,2023-12-22
4,2023-12-25


In [34]:
forecast_prophet = modelo_prophet.predict(future) #realizando previsão
forecast_prophet = forecast_prophet[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
forecast_prophet

,ds,yhat,yhat_lower,yhat_upper
0,2023-12-19,83.781422,69.828703,96.800579
1,2023-12-20,83.721192,69.718171,96.716286
2,2023-12-21,83.825829,70.334080,97.910627
3,2023-12-22,83.903620,70.565166,97.254243
4,2023-12-25,83.989852,69.767375,98.568387


In [35]:
#gerando linha contínua para banda de confiança (é preciso uma linha contínua para aplicar o preenchimento)
banda_sup_prophet = forecast_prophet['yhat_upper'].tolist() #limite superior
banda_inf_prophet = forecast_prophet['yhat_lower'].tolist() #limite inferior
banda_prophet = banda_sup_prophet + banda_inf_prophet[::-1] #concatenando banda superior e banda inferior invertida 
banda_prophet_index = forecast_prophet.ds.tolist()
banda_prophet_index = banda_prophet_index + banda_prophet_index[::-1] #concatenando datas em ordem crescente com ordem decrescente

In [36]:
#concatenando treino e teste
treino_px = treino_prophet.set_index('ds') 
treino_px = treino_px[(treino_px.index >= '2023-09-01') & (treino_px.index < '2023-11-18')]
teste_px = teste_prophet
base_prophet_px = pd.concat([teste_px, treino_px]).rename(columns = {'y': 'Preço'})
base_prophet_px.head()

,Preço
ds,
2023-12-26,80.23
2023-12-22,80.23
2023-12-21,80.73
2023-12-20,81.10
2023-12-19,79.82


In [37]:
forecast_prophet = pd.merge(forecast_prophet, teste_prophet, on='ds')
forecast_prophet = forecast_prophet.set_index('ds')
forecast_prophet

,yhat,yhat_lower,yhat_upper,y
ds,,,,
2023-12-19,83.781422,69.828703,96.800579,79.82
2023-12-20,83.721192,69.718171,96.716286,81.10
2023-12-21,83.825829,70.334080,97.910627,80.73
2023-12-22,83.903620,70.565166,97.254243,80.23


In [38]:
fig = px.line(
    base_prophet_px, 
    template=template,
    color_discrete_sequence=['#00008B']
)
fig.add_scatter(
    x=banda_prophet_index, 
    y=banda_prophet,
    name='Banda de confiança',
    mode='lines',
    line_color='#B22222',
    line_width=0,
    opacity=0.5,
    fill='toself'
)
fig.add_scatter(
    x=forecast_prophet.index, 
    y=forecast_prophet.yhat, 
    name='Preço previsto',
    marker=dict(color='#B22222')
)
fig.update_layout(
    title='Prophet',
    xaxis_title='Período',
    yaxis_title='Preço do Petróleo Brent',
    legend_title='Legenda',
    width=1600, 
    height=600,
    margin=dict(l=100, r=20, t=80, b=100)
)
fig.show()

### Gradient Boosting Regressor

**Preparando dados para o modelo Gradient Boosting Regressor**

In [39]:
df_gbr = df.sort_values('data').copy()
df_gbr.head()

,data,preco
11073,1987-05-20,18.63
11072,1987-05-21,18.45
11071,1987-05-22,18.55
11070,1987-05-25,18.60
11069,1987-05-26,18.63


In [40]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split


#criando lag features
for lag in range(1, 6):
    df_gbr[f'preco_lag_{lag}'] = df_gbr['preco'].shift(lag)

#excluindo os dados nulos gerados
df_gbr = df_gbr.dropna()

#preparando dados de treino e de teste
X = df_gbr[['preco_lag_1', 'preco_lag_2', 'preco_lag_3', 'preco_lag_4', 'preco_lag_5']].values #as features são o próprio preço com atraso
y = df_gbr['preco'].values #a target é o preço real atual

#separando dados de treino e de teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

#treinando modelo
modelo_gbr = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=0, loss='squared_error')
modelo_gbr.fit(X_train, y_train)

y_pred = modelo_gbr.predict(X_test)

In [41]:
df_gbr.tail()

,data,preco,preco_lag_1,preco_lag_2,preco_lag_3,preco_lag_4,preco_lag_5
4,2023-12-19,79.82,78.89,76.84,77.05,74.14,74.11
3,2023-12-20,81.10,79.82,78.89,76.84,77.05,74.14
2,2023-12-21,80.73,81.10,79.82,78.89,76.84,77.05
1,2023-12-22,80.23,80.73,81.10,79.82,78.89,76.84
0,2023-12-26,80.23,80.23,80.73,81.10,79.82,78.89


In [42]:
df_gbr.tail(1).drop(columns=['data','preco'])

,preco_lag_1,preco_lag_2,preco_lag_3,preco_lag_4,preco_lag_5
0,80.23,80.73,81.1,79.82,78.89


## Avaliação

### Função de validação

Erro absoluto percentual médio (MAPE) é uma métrica para avaliar modelos de regressão em machine learning. Um valor de MAPE baixo significa que as previsões estão próximas dos valores observados, enquanto um valor alto indica que as previsões estão longe dos valores observados (o modelo precisa ser melhorado).

$$\text{MAPE}(y, \hat{y}) = \frac{1}{N} \sum_{i=0}^{n - 1} \text{|} \frac{y_i - \hat{y}_i}{y_i} \text{|}$$

O WMAPE (Weighted Mean Absolute Percentage Error) é uma variação do MAPE que faz as somas dos erros percentuais e dos valores observados antes da divisão. O WAPE é outra maneira de escapar do problema de valores zero nos dados observados. Ele é outra alternativa recomendada quando a maioria de seus produtos não vendem em todos os períodos ou são vendidos em pouca quantidade. Quando um produto tem baixo volume de vendas, qualquer erro na previsão tem um impacto significativo na porcentagem de erro calculada pelo MAPE. Mesmo que o erro absoluto seja pequeno, ele será dividido por um valor muito baixo, resultando em uma porcentagem de erro elevada. Isso pode dar a impressão que a previsão é muito pior do que realmente é. Já no caso do WAPE, como a divisão é feita pela soma de todos os valores reais absolutos de uma vez, esse efeito é menor.

$$\text{WMAPE}(y, \hat{y}) = \frac{\sum_{i=1}^{n} \text{|} \hat{y}_i - {y_i} \text{|}}{\sum_{i=1}^{n} \text{|} {y}_i \text{|}}

In [43]:
#função para validação dos modelos
def wmape(y_true, y_pred):
    return np.abs(y_true - y_pred).sum() / np.abs(y_true).sum()

In [44]:
wmape_arima = wmape(forecast_arima['y'].values, forecast_arima['AutoARIMA'].values)
print(f'WAPE: {wmape_arima}')

WAPE: 0.027278076985454463


In [45]:
wmape_prophet = wmape(forecast_prophet['y'].values, forecast_prophet['yhat'].values)
print(f'WAPE: {wmape_prophet:.2%}')

WAPE: 4.15%


In [46]:
wmape_gbr = wmape(y_test, y_pred)
print(f'WAPE: {wmape_gbr:.2%}')

WAPE: 1.89%


**Exportando modelo**

In [47]:
import joblib
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA

df_statsforecast = df[['data', 'preco']].rename(columns={'data': 'ds', 'preco': 'y'})
df_statsforecast['unique_id'] = 'Preco'
df_statsforecast.dropna(inplace=True)

treino = df_statsforecast.loc[(df_statsforecast['ds'] >= '2000-01-01') & (df_statsforecast['ds'] <= df_statsforecast['ds'].loc[0])] #novos dados de treino
h = 5

modelo = StatsForecast(models=[AutoARIMA(season_length=5)], freq='B', n_jobs=-1)
modelo.fit(treino)
joblib.dump(modelo, 'modelos/auto_arima.joblib')

modelo = joblib.load('modelos/auto_arima.joblib')
forecast = modelo.predict(h=h, level=[90])
forecast = forecast[['ds', 'AutoARIMA']].reset_index(drop=True).rename(columns={'ds': 'data', 'AutoARIMA': 'preco_previsto'})
forecast['preco_previsto'] = [int(n * 100) / 100 for n in forecast['preco_previsto']]
forecast

,data,preco_previsto
0,2023-12-27,80.34
1,2023-12-28,80.34
2,2023-12-29,80.32
3,2024-01-01,80.32
4,2024-01-02,80.32


## Implementação